# Load libs

In [1]:
import os
import requests

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd 
import json 
import datetime
import re

pd.set_option('display.max_columns', None)

# Load .env & handle

In [69]:
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")

In [70]:
HANDLE = "@Sontungmtp"

**Các prefix phổ biến (không chính thức)**

```json
PREFIX_MAP = {
    # công khai
    "videos": "UULF",           # All videos (không phải uploads chronology như UU..., mà là section "Videos")
    "popular_videos": "UULP",   # Popular videos
    "live_streams": "UULV",     # Live streams
    "shorts": "UUSH",           # Short videos
    "popular_shorts": "UUPS",   # Popular short videos
    "popular_live": "UUPV",     # Popular live streams

    # members-only (thường không truy cập được)
    "members_videos": "UUMF",
    "members_all": "UUMO",
    "members_shorts": "UUMS",
    "members_live": "UUMV",
}
```

## 1) Lấy thông tin channel + uploads playlist

In [71]:
ch_res = requests.get(
    "https://www.googleapis.com/youtube/v3/channels",
    params={
        "part": "snippet,statistics,contentDetails",
        "forHandle": HANDLE,   # <— handle
        "key": API_KEY
    }
).json()

if not ch_res.get("items"):
    raise SystemExit("Không tìm thấy channel với handle này.")

ch_item = ch_res["items"][0]
channel_id = ch_item["id"]
uploads_pl = ch_item["contentDetails"]["relatedPlaylists"]["uploads"]

print("Channel ID:", channel_id)
print("Title:", ch_item["snippet"]["title"])
print("Subs:", ch_item["statistics"].get("subscriberCount"))

Channel ID: UClyA28-01x4z60eWQ2kiNbA
Title: Sơn Tùng M-TP Official
Subs: 11500000


In [72]:
ch_res

{'kind': 'youtube#channelListResponse',
 'etag': 'jpssRWOnvyIoo1CT9_xpvFKGvS8',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'GiPqX4TB2lGugLXxCKLPuk55u0o',
   'id': 'UClyA28-01x4z60eWQ2kiNbA',
   'snippet': {'title': 'Sơn Tùng M-TP Official',
    'description': 'Son Tung M-TP Official YouTube Channel - Property of Sơn Tùng M-TP\n\nMore information about Sơn Tùng M-TP: \nhttps://www.facebook.com/MTP.Fan\nhttps://www.instagram.com/sontungmtp\nhttps://twitter.com/sontungmtp777\nhttps://www.youtube.com/c/sontungmtp \n@Nhaccuatui: https://nhaccuatui.com/sontungmtp\n@Spotify: https://spoti.fi/2HPWs20\n@Itunes: https://apple.co/2rlSl3w\n\nMore about M-TP ENTERTAINMENT\nhttps://www.facebook.com/mtptown\nhttps://mtpentertainment.com\nhttps://twitter.com/mtpent_official\nhttps://www.instagram.com/mtpent_official',
    'customUrl': '@sontungmtp',
    'publishedAt': '2015-01-21T02:38:32Z',
    'thumbnails': {'default': {'url': 'https://yt

In [73]:
df_channel_dbt = pd.json_normalize(ch_res, sep="__")
df_channel_dbt

,kind,etag,items,pageInfo__totalResults,pageInfo__resultsPerPage
0,youtube#channelListResponse,jpssRWOnvyIoo1CT9_xpvFKGvS8,"[{'kind': 'youtube#channel', 'etag': 'GiPqX4TB...",1,5


In [74]:
# Trích xuất dữ liệu
channels = []
for item in ch_res.get("items", []):
    channels.append({
        "channel_id": item["id"],
        "title": item["snippet"]["title"],
        "description": item["snippet"]["description"],
        "customUrl": item["snippet"].get("customUrl"),
        "publishedAt": item["snippet"]["publishedAt"],
        "country": item["snippet"].get("country"),
        "thumbnail_high": item["snippet"]["thumbnails"]["high"]["url"],
        "uploads_playlist": item["contentDetails"]["relatedPlaylists"]["uploads"],
        "subscriberCount": int(item["statistics"].get("subscriberCount", 0)),
        "viewCount": int(item["statistics"].get("viewCount", 0)),
        "videoCount": int(item["statistics"].get("videoCount", 0)),
    })

# Đưa vào DataFrame
df_channels = pd.DataFrame(channels)
df_channels

,channel_id,title,description,customUrl,publishedAt,country,thumbnail_high,uploads_playlist,subscriberCount,viewCount,videoCount
0,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,Son Tung M-TP Official YouTube Channel - Prope...,@sontungmtp,2015-01-21T02:38:32Z,VN,https://yt3.ggpht.com/c-Z7mIlntSpG6VyQ5ZqaPggq...,UUlyA28-01x4z60eWQ2kiNbA,11500000,3403026164,199


## 2) Lấy ra playlists

In [87]:
# Lấy toàn bộ playlists của channel
pl_items = []
page_token = None

while True:
    pl_res = requests.get(
        "https://www.googleapis.com/youtube/v3/playlists",
        params={
            "part": "snippet,contentDetails",
            "channelId": channel_id,
            "maxResults": 50,
            "pageToken": page_token,
            "key": API_KEY
        }
    ).json()
    
    pl_items.extend(pl_res.get("items", []))
    page_token = pl_res.get("nextPageToken")
    if not page_token:
        break

print("Số playlists:", len(pl_items))

Số playlists: 31


In [76]:
pl_items[0]

{'kind': 'youtube#playlist',
 'etag': 'fcX3Bw03PVhI4qb0_t8EC4Izj7c',
 'id': 'PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20',
 'snippet': {'publishedAt': '2024-05-25T15:41:35.585403Z',
  'channelId': 'UClyA28-01x4z60eWQ2kiNbA',
  'title': 'SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU',
  'description': '',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg',
    'width': 480,
    'height': 360},
   'standard': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/sddefault.jpg',
    'width': 640,
    'height': 480},
   'maxres': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/maxresdefault.jpg',
    'width': 1280,
    'height': 720}},
  'channelTitle': 'Sơn Tùng M-TP Official',
  'localized': {'title': 'SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU',
   'description': ''

In [77]:
df_playlist_dbt = pd.json_normalize(pl_res, sep="__")
df_playlist_dbt.head(2)

,kind,etag,items,pageInfo__totalResults,pageInfo__resultsPerPage
0,youtube#playlistListResponse,uSHyWmCJ9gn3khMPKph8Da9orvY,"[{'kind': 'youtube#playlist', 'etag': 'fcX3Bw0...",31,50


In [78]:
df_playlist_dbt['items'][0]

[{'kind': 'youtube#playlist',
  'etag': 'fcX3Bw03PVhI4qb0_t8EC4Izj7c',
  'id': 'PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20',
  'snippet': {'publishedAt': '2024-05-25T15:41:35.585403Z',
   'channelId': 'UClyA28-01x4z60eWQ2kiNbA',
   'title': 'SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU',
   'description': '',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/sddefault.jpg',
     'width': 640,
     'height': 480},
    'maxres': {'url': 'https://i.ytimg.com/vi/abPmZCZZrFA/maxresdefault.jpg',
     'width': 1280,
     'height': 720}},
   'channelTitle': 'Sơn Tùng M-TP Official',
   'localized': {'title': 'SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐA

In [79]:
# phẳng JSON -> DataFrame
df = pd.json_normalize(pl_items, sep="__")

# chuẩn hóa tên cột chính
rename_map = {
    "id": "playlist_id",
    "snippet__channelId": "channel_id",
    "snippet__channelTitle": "channel_title",
    "snippet__title": "playlist_title",
    "snippet__description": "playlist_desc",
    "snippet__publishedAt": "published_at",
    "contentDetails__itemCount": "item_count",
    "snippet__thumbnails__default__url": "thumb_default",
    "snippet__thumbnails__medium__url": "thumb_medium",
    "snippet__thumbnails__high__url": "thumb_high",
}
df = df.rename(columns=rename_map)

# ép kiểu & parse thời gian
df["published_at"] = pd.to_datetime(df["published_at"], errors="coerce")
df["item_count"]   = pd.to_numeric(df["item_count"], errors="coerce").astype("Int64")

# chọn subset cột gọn gàng
df_playlists = df[
    ["playlist_id", "playlist_title", "playlist_desc",
     "published_at", "channel_id", "channel_title",
     "item_count", "thumb_default", "thumb_medium", "thumb_high"]
]

df_playlists.head(2)

,playlist_id,playlist_title,playlist_desc,published_at,channel_id,channel_title,item_count,thumb_default,thumb_medium,thumb_high
0,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU,,2024-05-25 15:41:35.585403+00:00,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,5,https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg
1,PLBcAa442MLApwqf4ZeQCurYFaDUEat3kI,SƠN TÙNG M-TP | CHÚNG TA CỦA TƯƠNG LAI,“Chúng Ta Của Tương Lai” là phần tiếp nối của ...,2024-03-01 05:21:49.725916+00:00,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,6,https://i.ytimg.com/vi/IpDNg7Xj2R4/default.jpg,https://i.ytimg.com/vi/IpDNg7Xj2R4/mqdefault.jpg,https://i.ytimg.com/vi/IpDNg7Xj2R4/hqdefault.jpg


## 3) Lấy ra Videos trong Playlist

In [81]:
channel_id

'UClyA28-01x4z60eWQ2kiNbA'

In [116]:

# --- 2. Lấy items (video) trong từng playlist ---
def fetch_playlist_items(playlist_id):
    items, page_token = [], None
    while True:
        res = requests.get(
            "https://www.googleapis.com/youtube/v3/playlistItems",
            params={
                "part": "snippet,contentDetails,status",
                "playlistId": playlist_id,
                "maxResults": 50,
                "pageToken": page_token,
                "key": API_KEY
            },
            timeout=30
        ).json()
        items.extend(res.get("items", []))
        page_token = res.get("nextPageToken")
        if not page_token:
            break
    return items

all_items = []
for pid in df_playlists["playlist_id"]:
    items = fetch_playlist_items(pid)
    for it in items:
        it["playlist_id"] = pid   # thêm id playlist cha để join
    all_items.extend(items)

print("Tổng số video trong tất cả playlists:", len(all_items))

df_playlist_items = pd.json_normalize(all_items, sep="__").rename(columns={"id": "playlistItem_id"})
df_playlist_items = df_playlist_items.rename(columns={
    "snippet__title": "video_title",
    "snippet__description": "video_desc",
    "snippet__publishedAt": "added_to_playlist_at",
    "contentDetails__videoPublishedAt": "video_published_at",
    "contentDetails__videoId": "video_id",
    "snippet__channelId": "channel_id",
    "snippet__channelTitle": "channel_title",
    "snippet__position": "position_in_playlist",
})
df_playlist_items["added_to_playlist_at"] = pd.to_datetime(df_playlist_items["added_to_playlist_at"], errors="coerce")
df_playlist_items["video_published_at"] = pd.to_datetime(df_playlist_items["video_published_at"], errors="coerce")

# --- Kết quả ---
print("Playlists shape:", df_playlists.shape)
print("Items shape:", df_playlist_items.shape)

Tổng số video trong tất cả playlists: 234
Playlists shape: (31, 10)
Items shape: (234, 33)


In [117]:
df_playlist_items.head(2)

,kind,etag,playlistItem_id,playlist_id,added_to_playlist_at,channel_id,video_title,video_desc,snippet__thumbnails__default__url,snippet__thumbnails__default__width,snippet__thumbnails__default__height,snippet__thumbnails__medium__url,snippet__thumbnails__medium__width,snippet__thumbnails__medium__height,snippet__thumbnails__high__url,snippet__thumbnails__high__width,snippet__thumbnails__high__height,snippet__thumbnails__standard__url,snippet__thumbnails__standard__width,snippet__thumbnails__standard__height,snippet__thumbnails__maxres__url,snippet__thumbnails__maxres__width,snippet__thumbnails__maxres__height,channel_title,snippet__playlistId,position_in_playlist,snippet__resourceId__kind,snippet__resourceId__videoId,snippet__videoOwnerChannelTitle,snippet__videoOwnerChannelId,video_id,video_published_at,status__privacyStatus
0,youtube#playlistItem,o6Nt501sR3uI9OxUx5BMFdn64uI,UExCY0FhNDQyTUxBcGhTNkF4b1VMM0dtZmNHOEpSeGYyMC...,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,2024-06-07 06:34:28+00:00,UClyA28-01x4z60eWQ2kiNbA,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,Hãy cùng thưởng thức ca khúc ĐỪNG LÀM TRÁI TIM...,https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg,120.0,90.0,https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg,320.0,180.0,https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg,480.0,360.0,https://i.ytimg.com/vi/abPmZCZZrFA/sddefault.jpg,640.0,480.0,https://i.ytimg.com/vi/abPmZCZZrFA/maxresdefau...,1280.0,720.0,Sơn Tùng M-TP Official,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,0,youtube#video,abPmZCZZrFA,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,abPmZCZZrFA,2024-06-08 13:00:01+00:00,public
1,youtube#playlistItem,GB7618pT8IqW0xoaimDJq3nhGXo,UExCY0FhNDQyTUxBcGhTNkF4b1VMM0dtZmNHOEpSeGYyMC...,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,2024-05-29 08:47:18+00:00,UClyA28-01x4z60eWQ2kiNbA,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,ĐỪNG LÀM TRÁI TIM ANH ĐAU\n❤️‍🩹\n20:00 | 08-06...,https://i.ytimg.com/vi/CQXQKr_3vKE/default.jpg,120.0,90.0,https://i.ytimg.com/vi/CQXQKr_3vKE/mqdefault.jpg,320.0,180.0,https://i.ytimg.com/vi/CQXQKr_3vKE/hqdefault.jpg,480.0,360.0,https://i.ytimg.com/vi/CQXQKr_3vKE/sddefault.jpg,640.0,480.0,https://i.ytimg.com/vi/CQXQKr_3vKE/maxresdefau...,1280.0,720.0,Sơn Tùng M-TP Official,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,1,youtube#video,CQXQKr_3vKE,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,CQXQKr_3vKE,2024-05-29 12:58:00+00:00,public


## 4) Lấy statistics của các videos

In [118]:
# 1) Lấy danh sách video_id duy nhất
list_video = df_playlist_items["video_id"].dropna().astype(str).unique().tolist()
print("Số video duy nhất:", len(list_video))

Số video duy nhất: 181


In [ ]:
# 2) Hàm gọi videos.list để lấy statistics (batch 50 ID/lần)
def fetch_videos_stats(video_ids, api_key, part="snippet,contentDetails,statistics,liveStreamingDetails"):
    all_items = []
    for i in range(0, len(video_ids), 50):
        batch_ids = ",".join(video_ids[i:i+50])
        res = requests.get(
            "https://www.googleapis.com/youtube/v3/videos",
            params={"part": part, "id": batch_ids, "key": api_key},
            timeout=30
        )
        res.raise_for_status()
        all_items.extend(res.json().get("items", []))
    if not all_items:
        return pd.DataFrame(columns=[
            "video_id","video_title","video_desc","video_published_at",
            "duration_iso8601","view_count","like_count","comment_count",
            "v_thumb_default","v_thumb_medium","v_thumb_high",
            "live_actual_start","live_actual_end","live_scheduled_start"
        ])

    df = pd.json_normalize(all_items, sep="__").rename(columns={"id": "video_id"})
    # chọn & chuẩn hoá cột
    rename_map = {
        "snippet__title": "video_title",
        "snippet__description": "video_desc",
        "snippet__publishedAt": "video_published_at",
        "contentDetails__duration": "duration_iso8601",
        "statistics__viewCount": "view_count",
        "statistics__likeCount": "like_count",
        "statistics__commentCount": "comment_count",
        "snippet__thumbnails__default__url": "v_thumb_default",
        "snippet__thumbnails__medium__url": "v_thumb_medium",
        "snippet__thumbnails__high__url": "v_thumb_high",
        "liveStreamingDetails__actualStartTime": "live_actual_start",
        "liveStreamingDetails__actualEndTime": "live_actual_end",
        "liveStreamingDetails__scheduledStartTime": "live_scheduled_start",
    }
    df = df.rename(columns=rename_map)

    # ép kiểu số
    for c in ["view_count","like_count","comment_count"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")

    # parse thời gian
    for c in ["video_published_at","live_actual_start","live_actual_end","live_scheduled_start"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)

    # đảm bảo đủ cột
    for col in ["video_id","video_title","video_desc","video_published_at",
                "duration_iso8601","view_count","like_count","comment_count",
                "v_thumb_default","v_thumb_medium","v_thumb_high",
                "live_actual_start","live_actual_end","live_scheduled_start"]:
        if col not in df.columns:
            df[col] = pd.NA

    return df[[
        "video_id","video_title","video_desc","video_published_at",
        "duration_iso8601","view_count","like_count","comment_count",
        "v_thumb_default","v_thumb_medium","v_thumb_high",
        "live_actual_start","live_actual_end","live_scheduled_start"
    ]]

In [113]:
all_items

[{'kind': 'youtube#playlistItem',
  'etag': 'o6Nt501sR3uI9OxUx5BMFdn64uI',
  'id': 'UExCY0FhNDQyTUxBcGhTNkF4b1VMM0dtZmNHOEpSeGYyMC4wOTA3OTZBNzVEMTUzOTMy',
  'snippet': {'publishedAt': '2024-06-07T06:34:28Z',
   'channelId': 'UClyA28-01x4z60eWQ2kiNbA',
   'title': 'SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OFFICIAL MUSIC VIDEO',
   'description': 'Hãy cùng thưởng thức ca khúc ĐỪNG LÀM TRÁI TIM ANH ĐAU ngay tại đây nhé: 👉🏻 👉🏻 👉🏻  https://vivienm.lnk.to/DLTTAD 💍❤️\u200d🩹🧩\n\n#DLTTAD #SonTungMTP #DungLamTraiTimAnhDau \n\n🚫🤲🏻♥️🙆🏻\u200d♂️😢\n\n▶ More information about Sơn Tùng M-TP: \nhttps://www.facebook.com/MTP.Fan\nhttps://www.instagram.com/sontungmtp\nhttps://www.youtube.com/sontungmtp\nhttps://www.tiktok.com/@tiger050794 \nhttps://twitter.com/sontungmtp777\n@Spotify: https://spoti.fi/2HPWs20\n@Itunes: https://apple.co/2rlSl3w\n\n▶More information about M-TP Talent:\nhttps://www.facebook.com/mtptalent\nhttps://www.instagram.com/mtptalent\nhttps://twitter.com/mtptalent\n\n▶ More about M-

In [97]:
# 3) Gọi API & merge vào df_items
df_video_stats = fetch_videos_stats(list_video, API_KEY)
print("df_video_stats shape:", df_video_stats.shape)

df_video_stats shape: (175, 14)


In [142]:
df_video_stats.head(2)

,video_id,video_title,video_desc,video_published_at,duration_iso8601,view_count,like_count,comment_count,v_thumb_default,v_thumb_medium,v_thumb_high,live_actual_start,live_actual_end,live_scheduled_start
0,abPmZCZZrFA,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,Hãy cùng thưởng thức ca khúc ĐỪNG LÀM TRÁI TIM...,2024-06-08 13:00:01+00:00,PT5M26S,156093693,1401306,251046,https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg,NaT,NaT,NaT
1,CQXQKr_3vKE,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,ĐỪNG LÀM TRÁI TIM ANH ĐAU\n❤️‍🩹\n20:00 | 08-06...,2024-05-29 12:58:00+00:00,PT22S,1772288,72234,5781,https://i.ytimg.com/vi/CQXQKr_3vKE/default.jpg,https://i.ytimg.com/vi/CQXQKr_3vKE/mqdefault.jpg,https://i.ytimg.com/vi/CQXQKr_3vKE/hqdefault.jpg,NaT,NaT,NaT


# Tổng hợp data

In [141]:
df_channels.head(2)

,channel_id,title,description,customUrl,publishedAt,country,thumbnail_high,uploads_playlist,subscriberCount,viewCount,videoCount
0,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,Son Tung M-TP Official YouTube Channel - Prope...,@sontungmtp,2015-01-21T02:38:32Z,VN,https://yt3.ggpht.com/c-Z7mIlntSpG6VyQ5ZqaPggq...,UUlyA28-01x4z60eWQ2kiNbA,11500000,3403026164,199


In [137]:
df_playlists.head(2)

,playlist_id,playlist_title,playlist_desc,published_at,channel_id,channel_title,item_count,thumb_default,thumb_medium,thumb_high
0,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU,,2024-05-25 15:41:35.585403+00:00,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,5,https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg
1,PLBcAa442MLApwqf4ZeQCurYFaDUEat3kI,SƠN TÙNG M-TP | CHÚNG TA CỦA TƯƠNG LAI,“Chúng Ta Của Tương Lai” là phần tiếp nối của ...,2024-03-01 05:21:49.725916+00:00,UClyA28-01x4z60eWQ2kiNbA,Sơn Tùng M-TP Official,6,https://i.ytimg.com/vi/IpDNg7Xj2R4/default.jpg,https://i.ytimg.com/vi/IpDNg7Xj2R4/mqdefault.jpg,https://i.ytimg.com/vi/IpDNg7Xj2R4/hqdefault.jpg


In [124]:
df_playlist_items.head(2)

,kind,etag,playlistItem_id,playlist_id,added_to_playlist_at,channel_id,video_title,video_desc,snippet__thumbnails__default__url,snippet__thumbnails__default__width,snippet__thumbnails__default__height,snippet__thumbnails__medium__url,snippet__thumbnails__medium__width,snippet__thumbnails__medium__height,snippet__thumbnails__high__url,snippet__thumbnails__high__width,snippet__thumbnails__high__height,snippet__thumbnails__standard__url,snippet__thumbnails__standard__width,snippet__thumbnails__standard__height,snippet__thumbnails__maxres__url,snippet__thumbnails__maxres__width,snippet__thumbnails__maxres__height,channel_title,snippet__playlistId,position_in_playlist,snippet__resourceId__kind,snippet__resourceId__videoId,snippet__videoOwnerChannelTitle,snippet__videoOwnerChannelId,video_id,video_published_at,status__privacyStatus
0,youtube#playlistItem,o6Nt501sR3uI9OxUx5BMFdn64uI,UExCY0FhNDQyTUxBcGhTNkF4b1VMM0dtZmNHOEpSeGYyMC...,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,2024-06-07 06:34:28+00:00,UClyA28-01x4z60eWQ2kiNbA,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,Hãy cùng thưởng thức ca khúc ĐỪNG LÀM TRÁI TIM...,https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg,120.0,90.0,https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg,320.0,180.0,https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg,480.0,360.0,https://i.ytimg.com/vi/abPmZCZZrFA/sddefault.jpg,640.0,480.0,https://i.ytimg.com/vi/abPmZCZZrFA/maxresdefau...,1280.0,720.0,Sơn Tùng M-TP Official,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,0,youtube#video,abPmZCZZrFA,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,abPmZCZZrFA,2024-06-08 13:00:01+00:00,public
1,youtube#playlistItem,GB7618pT8IqW0xoaimDJq3nhGXo,UExCY0FhNDQyTUxBcGhTNkF4b1VMM0dtZmNHOEpSeGYyMC...,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,2024-05-29 08:47:18+00:00,UClyA28-01x4z60eWQ2kiNbA,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,ĐỪNG LÀM TRÁI TIM ANH ĐAU\n❤️‍🩹\n20:00 | 08-06...,https://i.ytimg.com/vi/CQXQKr_3vKE/default.jpg,120.0,90.0,https://i.ytimg.com/vi/CQXQKr_3vKE/mqdefault.jpg,320.0,180.0,https://i.ytimg.com/vi/CQXQKr_3vKE/hqdefault.jpg,480.0,360.0,https://i.ytimg.com/vi/CQXQKr_3vKE/sddefault.jpg,640.0,480.0,https://i.ytimg.com/vi/CQXQKr_3vKE/maxresdefau...,1280.0,720.0,Sơn Tùng M-TP Official,PLBcAa442MLAphS6AxoUL3GmfcG8JRxf20,1,youtube#video,CQXQKr_3vKE,Sơn Tùng M-TP Official,UClyA28-01x4z60eWQ2kiNbA,CQXQKr_3vKE,2024-05-29 12:58:00+00:00,public


In [138]:
df_video_stats.head(2)

,video_id,video_title,video_desc,video_published_at,duration_iso8601,view_count,like_count,comment_count,v_thumb_default,v_thumb_medium,v_thumb_high,live_actual_start,live_actual_end,live_scheduled_start
0,abPmZCZZrFA,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,Hãy cùng thưởng thức ca khúc ĐỪNG LÀM TRÁI TIM...,2024-06-08 13:00:01+00:00,PT5M26S,156093693,1401306,251046,https://i.ytimg.com/vi/abPmZCZZrFA/default.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/mqdefault.jpg,https://i.ytimg.com/vi/abPmZCZZrFA/hqdefault.jpg,NaT,NaT,NaT
1,CQXQKr_3vKE,SƠN TÙNG M-TP | ĐỪNG LÀM TRÁI TIM ANH ĐAU | OF...,ĐỪNG LÀM TRÁI TIM ANH ĐAU\n❤️‍🩹\n20:00 | 08-06...,2024-05-29 12:58:00+00:00,PT22S,1772288,72234,5781,https://i.ytimg.com/vi/CQXQKr_3vKE/default.jpg,https://i.ytimg.com/vi/CQXQKr_3vKE/mqdefault.jpg,https://i.ytimg.com/vi/CQXQKr_3vKE/hqdefault.jpg,NaT,NaT,NaT


## Save data

In [143]:
df_channels.to_csv("data/channels.csv", index=False)
df_playlists.to_csv("data/playlists.csv", index=False)
df_playlist_items.to_csv("data/playlist_items.csv", index=False)
df_video_stats.to_csv("data/video_stats.csv", index=False)